In [29]:
import sys

sys.path.append("../")

import numpy as np
import pandas_ta as ta
import joblib

from terminal.Extensions.MT5Ext import MT5Ext

from sklearn.metrics import accuracy_score

In [30]:
# load, no need to initialize the loaded_rf
model = joblib.load("../models/WIN$N-2024_06-RandomForestClassifier.joblib")

In [31]:
def feature_target(file):
    # Carregar os dados
    ticks = np.load(file, allow_pickle=True)["ticks"]
    data = MT5Ext.create_ohlc_from_ticks(ticks, "5s")

    del ticks

    # Cálculo de indicadores
    data.ta.macd(append=True)
    data.dropna(inplace=True)

    # Pré-processamento de dados
    data["change"] = data["close"].pct_change()
    data["target"] = np.where(data["change"] > 0, 1, 0)
    data["target"] = data["target"].shift(-1)
    data.dropna(inplace=True)

    bought_data = data[data["close"] > data["open"]]
    sold_data = data[data["close"] < data["open"]]

    # Criar características e rótulos
    feature_columns = [
        "open",
        "close",
        "MACD_12_26_9",
        "MACDh_12_26_9",
        "MACDs_12_26_9",
    ]

    bought_features = bought_data[feature_columns]
    bought_target = bought_data["target"]

    sold_features = sold_data[feature_columns]
    sold_target = sold_data["target"]

    del data
    del bought_data
    del sold_data

    return bought_features, bought_target, sold_features, sold_target


def test(file):
    bought_features, bought_target, sold_features, sold_target = feature_target(file)

    # Avaliar o modelo
    y_pred = model.predict(bought_features)
    accuracy = accuracy_score(bought_target, y_pred)
    print(f"bought accuracy: {accuracy:.2f}")
    
    # Avaliar o modelo
    y_pred = model.predict(sold_features)
    accuracy = accuracy_score(sold_target, y_pred)
    print(f"sold accuracy: {accuracy:.2f}")

In [32]:
test("./data/ticks_WIN$N_2024_7.npz")

bought accuracy: 0.60
sold accuracy: 0.54
